In [ ]:
#Initial try, didnt work

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import h5py

#gets the data file from my google drive and saves it to a variable
data = h5py.File("//content//drive//MyDrive//Rocket Data//Cardiff_5_HOTFIRE.h5")

#instantiate a dataframe as a variable
df = pd.DataFrame()

# Go though each channel in the dataset
for i in data["channels"]:
  # Access the time data for the channel
  time_data = data["channels"][f"{i}"]["time"][()]

  # Access the data for the channel
  channel_data = data["channels"][f"{i}"]["data"][()]

  # Create a pandas DataFrame with 'time' and the channel data
  df = pd.DataFrame({'time': time_data, 'data': channel_data})

  # Define the filename for the CSV
  filename = f"{i}.csv"

  # Save the DataFrame to a CSV file
  df.to_csv(filename, index=False)

  print(f"Exported channel {i} with time data to '{filename}'")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         time
0        0.00
1        0.01
2        0.02
3        0.03
4        0.04
...       ...
10042  100.42
10043  100.43
10044  100.44
10045  100.45
10046  100.46

[10047 rows x 1 columns]


ValueError: unbounded slice

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import h5py

#gets the file from my drive and saves it to a variable
data = h5py.File("/content/drive/MyDrive/Rocket Data/Cardiff_5_HOTFIRE.h5")

#instantiates a dictionary for the data, making the first entry the time data
data_array = {"Time" : data["channels"]["6AX_LOAD_CELL_FX"]["time"][()]}

#goes through each channel and adds the data to the dictionary
for i in data["channels"]:
  data_array[f"{i}"] = data["channels"][f"{i}"]["data"][()]

#creates a pandas dataframe with the created dictionary
df = pd.DataFrame(data_array)

#saves the dataframe as a CSV File
df.to_csv("Cardiff_5_hotfire.csv", index=False)

Mounted at /content/drive


In [ ]:
#generate animated graph
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import pandas as pd
import h5py
import os

data = h5py.File("/content/drive/MyDrive/Rocket Data/Cardiff_5_HOTFIRE.h5")

data_array = {
    "Time" : data["channels"]["6AX_LOAD_CELL_FX"]["time"][()],
    "Thrust" : data["channels"]["THRUST_STAND_LC1_CALIBRATED"]["data"][()],
    "Propellant Flow Rate" : data["channels"]["NITROUS_FT_1"]["data"][()]
    }

df = pd.DataFrame(data_array)

# Filter data for the time period 109 - 121
df_filtered = df[(df['Time'] >= 72) & (df['Time'] <= 92)].reset_index(drop=True)


# Create a directory to save the frames
if not os.path.exists("frames"):
    os.makedirs("frames")

# Generate frames for the Thrust plot
fig, ax = plt.subplots()
ax.set_xlabel("Time")
ax.set_ylabel("Thrust")
ax.set_title("Thrust vs Time (72s - 92s)")
ax.set_xlim(72, 92) # Set x-axis limits for consistent plotting
ax.set_ylim(df_filtered["Thrust"].min(), df_filtered["Thrust"].max()) # Set y-axis limits based on filtered data

for i in range(len(df_filtered)):
    ax.plot(df_filtered["Time"][:i], df_filtered["Thrust"][:i], color='blue')
    plt.savefig(f"frames/thrust_frame_{i:04d}.png")
    ax.lines[-1].remove() # Remove the line to draw the next frame

plt.close(fig)

# Generate frames for the Propellant Flow Rate plot
fig, ax = plt.subplots()
ax.set_xlabel("Time")
ax.set_ylabel("Propellant Flow Rate")
ax.set_title("Propellant Flow Rate vs Time (72s - 92s)")
ax.set_xlim(72, 92) # Set x-axis limits for consistent plotting
ax.set_ylim(df_filtered["Propellant Flow Rate"].min(), df_filtered["Propellant Flow Rate"].max()) # Set y-axis limits based on filtered data


for i in range(len(df_filtered)):
    ax.plot(df_filtered["Time"][:i], df_filtered["Propellant Flow Rate"][:i], color='red')
    plt.savefig(f"frames/flow_rate_frame_{i:04d}.png")
    ax.lines[-1].remove() # Remove the line to draw the next frame

plt.close(fig)

print("Generated frames for animation in the 'frames' directory.")

Mounted at /content/drive
Generated frames for animation in the 'frames' directory.


In [ ]:
# Install ffmpeg
!apt-get update && apt-get install ffmpeg -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,807 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://archive

In [ ]:
# Create video for Thrust animation
!ffmpeg -r 10 -i frames/thrust_frame_%04d.png -c:v libx264 -vf "fps=10,format=yuv420p" thrust_animation.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# Create video for Propellant Flow Rate animation
!ffmpeg -r 10 -i frames/flow_rate_frame_%04d.png -c:v libx264 -vf "fps=10,format=yuv420p" flow_rate_animation.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab